In [32]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
size = 200000

In [33]:
train_url = '/kaggle/input/mydataset/train_corr.csv'
test_url = '/kaggle/input/mydataset/test_corr.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [34]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [35]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [45]:
from sklearn.model_selection import train_test_split
MAX_FEATURES = 10000
EMBED_SIZE = 300
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(df_train['Comment'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['Comment'])
list_tokenized_test = tokenizer.texts_to_sequences(df_test['Comment'])
#RNN_CELL_SIZE = 32
RNN_CELL_SIZE = 64
MAX_LEN = 75   # Since our mean length is 56.6
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN)
X_test = pad_sequences(list_tokenized_test,maxlen=MAX_LEN)
y_train = df_train['Error']
y_test = df_test['Error']

In [46]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [47]:
sequence_input = Input(shape=(MAX_LEN,), dtype="int32")
embedded_sequences = Embedding(MAX_FEATURES, EMBED_SIZE)(sequence_input)

In [48]:
lstm = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences = True), name="bi_lstm_0")(embedded_sequences)# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)

In [49]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(10)(lstm, state_h)
#dense1 = Dense(20, activation="relu")(context_vector)
dense1 = Dense(32, activation="relu")(context_vector)
#dropout = Dropout(0.05)(dense1)
dropout = Dropout(0.02)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = keras.Model(inputs=sequence_input, outputs=output)

In [52]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

In [55]:
BATCH_SIZE = 128
EPOCHS = 20
history = model.fit(X_train,y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.2)

Epoch 1/20
126/126 [==============================] - 18s 143ms/step - loss: 0.0495 - tp: 8298.0000 - fp: 164.0000 - tn: 7475.0000 - fn: 129.0000 - accuracy: 0.9818 - precision: 0.9806 - recall: 0.9847 - auc: 0.9976 - val_loss: 2.1261 - val_tp: 2298.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 1719.0000 - val_accuracy: 0.5721 - val_precision: 1.0000 - val_recall: 0.5721 - val_auc: 0.0000e+00
Epoch 2/20
126/126 [==============================] - 19s 150ms/step - loss: 0.0568 - tp: 8267.0000 - fp: 173.0000 - tn: 7466.0000 - fn: 160.0000 - accuracy: 0.9793 - precision: 0.9795 - recall: 0.9810 - auc: 0.9974 - val_loss: 2.3466 - val_tp: 2222.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 1795.0000 - val_accuracy: 0.5531 - val_precision: 1.0000 - val_recall: 0.5531 - val_auc: 0.0000e+00
Epoch 3/20
126/126 [==============================] - 18s 139ms/step - loss: 0.0475 - tp: 8284.0000 - fp: 120.0000 - tn: 7519.0000 - fn: 143.0000 - accuracy: 0.9836 - precision: 0.9857 -

In [56]:
pred = (model.predict(X_test) > 0.5).astype("int32")

157/157 [==============================] - 3s 16ms/step


In [57]:
pred = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))

157/157 [==============================] - 3s 17ms/step
              precision    recall  f1-score   support

     Correct       0.53      0.54      0.53      1910
   Incorrect       0.71      0.71      0.71      3112

    accuracy                           0.64      5022
   macro avg       0.62      0.62      0.62      5022
weighted avg       0.64      0.64      0.64      5022



In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))

              precision    recall  f1-score   support

     Correct       0.53      0.55      0.54      1910
   Incorrect       0.72      0.70      0.71      3112

    accuracy                           0.65      5022
   macro avg       0.63      0.63      0.63      5022
weighted avg       0.65      0.65      0.65      5022



In [54]:
pred = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))

157/157 [==============================] - 4s 16ms/step
              precision    recall  f1-score   support

     Correct       0.53      0.55      0.54      1910
   Incorrect       0.72      0.70      0.71      3112

    accuracy                           0.65      5022
   macro avg       0.63      0.63      0.63      5022
weighted avg       0.65      0.65      0.65      5022



In [ ]:
pred = (model.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))